In [2]:
  #!/usr/bin/env python

try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as nc
from gi.repository import NumCosmoMath as ncm
import numpy as np
ncm.cfg_init()

cosmo = nc.HICosmo.new_from_name (nc.HICosmo, "NcHICosmoDEXcdm")
primo = nc.HIPrimPowerLaw.new ()

cosmo.add_submodel(primo)

cosmo.props.Omegab  = 0.048254
cosmo.props.Omegac  = 0.30711 - 0.048254
cosmo.props.H0 = 67.7
primo.props.n_SA = 0.96
h =cosmo.props.H0/100

dist = nc.Distance.new (2.0)

initialize the `ClusterAbundance` object

In [3]:
#First we need to define the mulltiplicity function here we will use the tinker
mulf = nc.MultiplicityFuncTinker.new()
mulf.set_mdef(nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
#Second we need to construct a filtered power spectrum 

#Here we are using the EH transfer function and ml power spectrum with a tophat filter
tf = nc.TransferFunc.new_from_name('NcTransferFuncEH')

psml = nc.PowspecMLTransfer.new(tf)
psml.require_kmin (1.0e-6)
psml.require_kmax (1.0e3)

psf = ncm.PowspecFilter.new(psml,ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()
psf.prepare(cosmo)
mfunc_oa = ncm.ObjArray.new()
mfunc_sigma8 = ncm.MSetFuncList.new('NcHICosmo:sigma8',psf)
mfunc_oa.add(mfunc_sigma8)
hmf =  nc.HaloMassFunction.new(dist,psf,mulf)


ca = nc.ClusterAbundance.new(hmf,None)
mset = ncm.MSet.new_array([cosmo])
mset.pretty_log()
cosmo.sigma8(psf)

#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.7                [FIXED]
#   -     Omegac[01]:  0.258856            [FIXED]
#   -     Omegax[02]:  0.7                 [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.048254            [FIXED]
#   -          w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#----------------------------------------------------------------------------------
# Model parameters
#   - ln10e10ASA[00]:  3.179               [FIXED]
#   - T_SA_ratio[01]:  0.2               

0.8884898175540845

# Binned approach
Compute the 2D integral

$$
N_{\alpha\beta}^{\rm predicted} = \Omega_s\int_{z_\alpha}^{z_{\alpha + 1}} dz\int_{\log_{10}M_\beta}^{\log_{10}M_{\beta + 1}}\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}d\log_{10}M
$$

In [ ]:
ncdata = nc.DataClusterNCount.new(ca)
z_nodes = ncm.Vector.new_array(np.linspace(0.25, 1.25, 16))
lnM_nodes = ncm.Vector.new_array(np.linspace(14, 14.8, 11))
ncdata.set_bin_by_nodes(z_nodes,lnM_nodes)

rng = ncm.RNG.pool_get("example_ca_sampling")
ncdata.init_from_sampling(mset, 0.25*4*np.pi, rng)